In [1]:
# Set up the base template
template = """
WHO YOU ARE:
you are a full stack engineer.
you are an expert to write down a code script.
you can make a beckend server code and front end code.
you are an expert for any language.

WHICH TOOLS YOU CAN USE :
These are the set of tools you can use.
If you think it is a proper tool to solve your problem, whatever you can use.

tools:
{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
please follow the input schema of tools.

HOW YOU MUST ANSWER :
You MUST follow this answering template.
Your answer must be in this shape.

WhatYouDid: what you did in the just previous step.
Planing: Make a plan what will you do based on the previous step. Each plan has a simple task. Anyone should be able to achieve the purpose if they follow your plan.
Understanding: Understand the 'Observation' or 'Purpose' from previous step.
NextStep: Write a simple task for just a next step.
Action:
```
$JSON_BLOB
```
Observation: this is the result of the action.
... (this WhatYouDid/Understanding/NextStep/Action/Action Input/Observation can repeat N times)
Understanding: I know what to respond
Action:
```
{{

  "action": "Final Answer",

  "action_input": "Final response to human"

}}
```

Now I need your answer.
Begin!

Purpose: {input}
WhatYouDid: {agent_scratchpad}
"""

In [2]:
from typing import List

from langchain.agents import Tool
from langchain.prompts import StringPromptTemplate


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nUnderstanding: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [3]:
from custom_tools.tools import (
    terminal_tool,
    search_url_tool,
    request_url_tool,
    readfile_tool,
    savefile_tool,
)
from langchain.tools import StructuredTool

tools = [
    StructuredTool.from_function(terminal_tool),
    StructuredTool.from_function(search_url_tool),
    StructuredTool.from_function(request_url_tool),
    StructuredTool.from_function(savefile_tool),
    StructuredTool.from_function(readfile_tool),
    StructuredTool.from_function(request_url_tool),
]

In [4]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [5]:
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.agents import AgentOutputParser
from typing import Union
import re

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"action\s*\d*\s*:(.*?)\naction\s*\d*\s*input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: \n`{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [6]:
output_parser = CustomOutputParser()

In [7]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")


/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
from langchain.chains import LLMChain

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
from langchain.agents import LLMSingleActionAgent
from langchain.agents.output_parsers import JSONAgentOutputParser


tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=JSONAgentOutputParser(), #output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=8)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=64,
    handle_parsing_errors=True
)

In [12]:
purpose_prompt = (
    "The end purpose is make a web service with UI."
    "Make a project directory to save your result."
    "The background of the webpage is pastel yellow."
    "The canvas is at the center of webpage."
    "The save button is under the canvas."
    "The reset button is under the canvas to reset the canvas."
    "The user can draw lines on a canvas and save the picture with save button."
    "If user save a picture, server makes an image file and name it arbitrary words."
    "The list of saved picture is under the canvas on line by line."
    "Webpage can also redrects to one of the saved pictures when user click it's name in list."
    # "Make test scripts for each function you write in codes."
    # "Run all the test till all the test are passed."
    "After all, Run it to get the webpage."
    "Test the webpage is running well or not with terminal commands. It should not be '404 not found'."
    "Test saving and loading image with your server."
    "Use port 3000."
    # "Follow these steps:"
    # "1. Make a directory to save your result. If it is already exist, go to the next step."
    # "2. Save files of runnable codes in the directory of first step for backend server with python."
    # "3. Save files of runnable codes in the directory of first step for UI"
    # "4. Run it to get the webpage which show the step3 and perform step 2."
    # "5. Check the webpage is running well or not with terminal commands. It should not be '404 not found'."
    # # "6. Save a random salt-and-papper picture by this webpage and check the picture is openable"
    # "Perfomance Check List:"
    # "1. saved picture list"
    # "2. redirection to saved picture"
    # "3. drawing canvas"
    # "4. save button"
    # "5. reset button"
)


In [13]:
agent_executor.run(purpose_prompt)

/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
WhatYouDid: Received the task to create a web service with a UI that allows users to draw on a canvas, save the drawing, and view a list of saved drawings.
Planning: Break down the task into smaller, manageable steps:
1. Set up a project directory.
2. Create a backend server.
3. Design and implement the frontend UI.
4. Implement canvas drawing functionality.
5. Implement saving and listing of images.
6. Implement redirection to saved images.
7. Test the web service.
8. Ensure the server is running on port 3000 and is accessible.
9. Test saving and loading images.

Understanding: The first step is to create a project directory to organize the files and code for the web service.

NextStep: Create a project directory for the web service.

Action:
```
{
  "action": "terminal_tool",
  "action_input": {
    "command": "mkdir web_service_project && cd web_service_project",
    "timeout": 5
  }
}
```


Observation:STDOUT :  exit_code: 0 | STDERR :  exit_

"The web service with UI has been set up and tested. The server can save images and list saved images. Due to the limitations of the tools provided, persistent server testing and full end-to-end testing could not be performed. However, the server's response to the save request and the contents of the 'saved_images' directory indicate that the functionality is working as expected."